まずトレーニングデータとテストデータを用意する。
文字列のセルはそのままだと処理できないので、どうしよう

In [5]:
import pandas as pd
import numpy as np

raw_train = pd.read_csv(filepath_or_buffer="./train.csv")
raw_test = pd.read_csv(filepath_or_buffer="./test.csv")

raw_train["is_test"] = False
raw_test["is_test"] = True

all_data = pd.concat((raw_train, raw_test))

In [6]:
all_data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,is_test
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,False
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,False
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,False
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,False
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,False


In [18]:
# Refer to: [Titanic with Keras | Kaggle](https://www.kaggle.com/cstahl12/titanic-with-keras)

def get_title_last_name(name):
    full_name = name.str.split(', ', n=0, expand=True)
    # last_name = full_name[0] # last_name は使っていない模様
    titles = full_name[1].str.split('.', n=0, expand=True)
    titles = titles[0]
    return(titles)

def get_titles_from_names(df):
    """
    """
    df['Title'] = get_title_last_name(df['Name'])
    df = df.drop(['Name'], axis=1)
    return(df)

def get_cabin_letter(df):
    df['Cabin'].fillna('Z', inplace=True) # Cabin が空欄の行は "Z" で埋める
    df['Cabin_letter'] = df['Cabin'].str[0] # ?例えば "C91" ならば "C" でまとめてしまっても問題ないということ？なぜ？
    return(df)

def get_dummy_cats(df):
    return(pd.get_dummies(df, columns=['Title', 'Pclass', 'Sex', 'Embarked',
                                       'Cabin', 'Cabin_letter'])) # なんだこの便利メソッドは！

def process_data(df):
    # preprocess titles, cabin, embarked
    df = get_titles_from_names(df)
    df['Embarked'].fillna('S', inplace=True) # よくよくデータを見ると、Embarkedが空欄の行がある
    df = get_cabin_letter(df)
    
    # drop remaining features
    df = df.drop(['Ticket', 'Fare'], axis=1) # ?なぜdropする？
    
    # create dummies for categorial features
    df = get_dummy_cats(df)
    
    return(df)

proc_data = process_data(all_data)

proc_train = proc_data[proc_data["is_test"] == False]
proc_eval = proc_data[proc_data["is_test"] == False]
proc_test = proc_data[proc_data["is_test"] == True]

In [58]:
X = proc_train.drop(["Survived", "is_test"], axis=1)
y = pd.get_dummies(proc_train["Survived"]) # ?なぜこうする？

In [59]:
X["Age"].fillna(20.0, inplace=True)

Keras の設定

In [60]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# create model
model = Sequential()
model.add(Dense(input_dim=X.shape[1], units=128,
                 kernel_initializer='normal', bias_initializer='zeros'))
model.add(Activation('relu'))

for i in range(0, 15):
    model.add(Dense(units=128, kernel_initializer='normal',
                     bias_initializer='zeros'))
    model.add(Activation('relu'))
    model.add(Dropout(.40))

model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X.values, y.values,
          validation_split=0.3,
          epochs=20,
          batch_size=128)

Train on 623 samples, validate on 268 samples
Epoch 1/20
623/623 [==============================] - 2s 4ms/step - loss: 0.6910 - acc: 0.5778 - val_loss: 0.6839 - val_acc: 0.6418
Epoch 2/20
623/623 [==============================] - 0s 173us/step - loss: 0.6844 - acc: 0.6051 - val_loss: 0.6709 - val_acc: 0.6418
Epoch 3/20
623/623 [==============================] - 0s 182us/step - loss: 0.6768 - acc: 0.6051 - val_loss: 0.6595 - val_acc: 0.6418
Epoch 4/20
623/623 [==============================] - 0s 178us/step - loss: 0.6738 - acc: 0.6051 - val_loss: 0.6560 - val_acc: 0.6418
Epoch 5/20
623/623 [==============================] - 0s 185us/step - loss: 0.6741 - acc: 0.6051 - val_loss: 0.6546 - val_acc: 0.6418
Epoch 6/20
623/623 [==============================] - 0s 196us/step - loss: 0.6717 - acc: 0.6051 - val_loss: 0.6554 - val_acc: 0.6418
Epoch 7/20
623/623 [==============================] - 0s 187us/step - loss: 0.6710 - acc: 0.6051 - val_loss: 0.6557 - val_acc: 0.6418
Epoch 8/20
623/623

In [69]:
X_test = proc_test.drop(["Survived", "is_test"], axis=1)

p_survived = model.predict_classes(X_test.values, batch_size=128)

p_survived

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,